In [ ]:
# Pandas ve Numpy yüklemek için
!pip install pandas numpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from tqdm import tqdm  # İlerleme çubuğu için

# Grafik stili (Daha profesyonel görünüm için)
sns.set(style="whitegrid")

# ==========================================
# 1. VERİ YÜKLEME (Sudenaz'ın Dosyaları)
# ==========================================
# Not: Notebook 'notebooks/' klasöründe olduğu için veriler bir üst dizindedir ('../')
try:
    train_df = pd.read_csv('../train_list.csv')
    val_df = pd.read_csv('../val_list.csv')
    test_df = pd.read_csv('../test_list.csv')
    print("✅ Dosyalar başarıyla yüklendi.")
except FileNotFoundError:
    print("❌ HATA: CSV dosyaları bulunamadı! Lütfen dosya yollarını kontrol et.")

# Etiketleri daha anlaşılır yapalım (0 -> NORMAL, 1 -> PNEUMONIA)
label_map = {0: 'NORMAL', 1: 'PNEUMONIA'}
train_df['class_name'] = train_df['label'].map(label_map)

print(f"Eğitim Seti Boyutu: {len(train_df)}")
print(f"Doğrulama Seti Boyutu: {len(val_df)}")
print(f"Test Seti Boyutu: {len(test_df)}")

# ==========================================
# 2. SORU 1: "KAÇ TANE NORMAL, KAÇ TANE ZATÜRRE VAR?"
# ==========================================
# Bu kısım Class Imbalance (Sınıf Dengesizliği) problemini gösterir.

plt.figure(figsize=(12, 5))

# Sol taraf: Çubuk Grafik
plt.subplot(1, 2, 1)
ax = sns.countplot(x='class_name', data=train_df, palette='viridis')
plt.title('Eğitim Seti Sınıf Dağılımı', fontsize=14)
plt.xlabel('Sınıf')
plt.ylabel('Resim Sayısı')
# Sütunların üzerine sayıları yazdıralım
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + 0.35, p.get_height() + 50))

# Sağ taraf: Pasta Grafik (Oransal Gösterim)
plt.subplot(1, 2, 2)
train_df['class_name'].value_counts().plot.pie(autopct='%1.1f%%', colors=['#ff9999','#66b3ff'], startangle=90)
plt.title('Dengesizlik Oranı', fontsize=14)
plt.ylabel('')

plt.tight_layout()
plt.show()

# Raporlama
normal_count = len(train_df[train_df['label'] == 0])
pneumonia_count = len(train_df[train_df['label'] == 1])
print(f"📊 RAPOR: Zatürre vakaları, Normal vakaların {pneumonia_count / normal_count:.1f} katı!")
if pneumonia_count > normal_count * 1.5:
    print("⚠️ UYARI: Ciddi sınıf dengesizliği var. Eğitimde 'Class Weights' veya 'Augmentation' kullanılmalı.")

# ==========================================
# 3. SORU 2: "RESİMLERİN PARLAKLIKLARI NASIL?"
# ==========================================
# Neden? Bazen röntgen cihazları farklı ayarlarda çalışır. 
# Model hastalığı değil de "daha parlak resmi" öğrenmeye çalışmasın diye bunu kontrol ederiz.

def calculate_brightness(df, root_dir='../'):
    brightness_values = []
    print("\nResim parlaklıkları hesaplanıyor... (Bu işlem biraz sürebilir)")
    
    # tqdm ile ilerleme çubuğu gösterelim
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Dosya yolunu oluştur (Windows/Mac uyumu için os.path kullanıyoruz)
        img_path = os.path.join(root_dir, row['file_path'])
        
        # Resmi gri tonlamalı oku
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is not None:
            # Ortalama piksel değerini al (0=Siyah, 255=Beyaz)
            avg_brightness = np.mean(img)
            brightness_values.append(avg_brightness)
        else:
            # Eğer resim bulunamazsa (örn: yol hatası) None ekle
            brightness_values.append(None)
            
    return brightness_values

# Sadece Train seti için hesaplayalım
train_df['brightness'] = calculate_brightness(train_df)

# Varsa hatalı/okunamayan dosyaları temizle
plot_df = train_df.dropna(subset=['brightness'])

# Görselleştirme
plt.figure(figsize=(12, 6))
sns.histplot(data=plot_df, x='brightness', hue='class_name', kde=True, element="step", bins=50, palette='magma')
plt.title('Sınıflara Göre Parlaklık Dağılımı (Histogram)', fontsize=15)
plt.xlabel('Ortalama Parlaklık (0=Karanlık, 255=Parlak)')
plt.ylabel('Frekans')
plt.axvline(x=plot_df['brightness'].mean(), color='red', linestyle='--', label='Genel Ortalama')
plt.legend()
plt.show()

# İstatistiksel Sonuçlar
print("\n💡 PARLAKLIK ANALİZİ SONUCU:")
print(plot_df.groupby('class_name')['brightness'].describe()[['mean', 'std', 'min', 'max']])